## Andrej Karpathy's GPT video code run

In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [3]:
with open("input.txt", "r", encoding="utf-8") as f:
  text = f.read()

In [4]:
# First 300 letters in the shakespeare text
print(text[:300])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us


# Create Basic Character Level Encoding

In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print("vocab size:", vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocab size: 65


In [6]:
stoi = {ch : i for i, ch in enumerate(chars)}
itos = {i : ch for i, ch in enumerate(chars)}

encode = lambda s : [stoi[ch] for ch in s]
decode = lambda l : ''.join(itos[i] for i in l)

print(encode("Hello World"))
print(decode(encode("Hello World")))

[20, 43, 50, 50, 53, 1, 35, 53, 56, 50, 42]
Hello World


In [7]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape)
print(data[:500])

torch.Size([1115393])
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
        47, 59, 57

In [8]:
n = int(0.9 * len(data))
train_data = data[:n] 
valid_data = data[n:]
print(train_data.shape)
print(valid_data.shape)

torch.Size([1003853])
torch.Size([111540])


In [9]:
BLOCK_SIZE = 8
print(train_data[:BLOCK_SIZE + 1])

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])


In [16]:
x = train_data[:BLOCK_SIZE]
y = train_data[1:BLOCK_SIZE + 1] # y is offset by 1 index to the right of x 

for i in range(BLOCK_SIZE):
  context = x[:i + 1]
  target = y[i]

  print(f"When input is {context.tolist()}, the output is {target}")

# From the block size of 8, you get block_size numbered training examples

When input is [18], the output is 47
When input is [18, 47], the output is 56
When input is [18, 47, 56], the output is 57
When input is [18, 47, 56, 57], the output is 58
When input is [18, 47, 56, 57, 58], the output is 1
When input is [18, 47, 56, 57, 58, 1], the output is 15
When input is [18, 47, 56, 57, 58, 1, 15], the output is 47
When input is [18, 47, 56, 57, 58, 1, 15, 47], the output is 58


In [21]:
torch.manual_seed(1337)
BATCH_SIZE = 4
BLOCK_SIZE = 8

def get_batch(split):
  data = train_data if split == "train" else valid_data
  ix = torch.randint(len(data) - BLOCK_SIZE, (BATCH_SIZE, )) # Initiate random starting point in data of length BATCH_SIZE
  x = torch.stack([data[i:BLOCK_SIZE+i] for i in ix]) # Stack BLOCK_SIZE list from the start of ix[i] to ix[i] + BLOCK_SIZE (appending encoded values)
  y = torch.stack([data[i+1:BLOCK_SIZE+i+1] for i in ix]) # Stack BLOCK_SIZE list, similar to x, but with offset of 1
  return x, y, ix

xb, yb, _ = get_batch("train")
print("context")
print(xb.shape)
print(xb)

print("target")
print(yb.shape)
print(yb)
print("----------------------------")

for b in range(BATCH_SIZE):
  for t in range(BLOCK_SIZE):
    context = xb[b, :t+1]
    target = yb[b, t]
    print(f"when the input is {context.tolist()} the target is {target}")
  print('\n')

context
torch.Size([4, 8])
tensor([[53, 59,  6,  1, 58, 56, 47, 40],
        [49, 43, 43, 54,  1, 47, 58,  1],
        [13, 52, 45, 43, 50, 53,  8,  0],
        [ 1, 39,  1, 46, 53, 59, 57, 43]])
target
torch.Size([4, 8])
tensor([[59,  6,  1, 58, 56, 47, 40, 59],
        [43, 43, 54,  1, 47, 58,  1, 58],
        [52, 45, 43, 50, 53,  8,  0, 26],
        [39,  1, 46, 53, 59, 57, 43,  0]])
----------------------------
when the input is [53] the target is 59
when the input is [53, 59] the target is 6
when the input is [53, 59, 6] the target is 1
when the input is [53, 59, 6, 1] the target is 58
when the input is [53, 59, 6, 1, 58] the target is 56
when the input is [53, 59, 6, 1, 58, 56] the target is 47
when the input is [53, 59, 6, 1, 58, 56, 47] the target is 40
when the input is [53, 59, 6, 1, 58, 56, 47, 40] the target is 59


when the input is [49] the target is 43
when the input is [49, 43] the target is 43
when the input is [49, 43, 43] the target is 54
when the input is [49, 43, 

# Bigram Language Model